# **3103. Find Trending Hashtags II**

**Table: Tweets**
``` sql
+-------------+---------+
| Column Name | Type    |
+-------------+---------+
| user_id     | int     |
| tweet_id    | int     |
| tweet_date  | date    |
| tweet       | varchar |
+-------------+---------+
```
tweet_id is the primary key (column with unique values) for this table.

Each row of this table contains user_id, tweet_id, tweet_date and tweet.

It is guaranteed that all tweet_date are valid dates in February 2024.

Write a solution to find the top 3 trending hashtags in February 2024. Every tweet may contain several hashtags.

Return the result table ordered by count of hashtag, hashtag in descending order.

The result format is in the following example.

 

**Example 1:**

**Input:**

**Tweets table:**
``` sql
+---------+----------+------------------------------------------------------------+------------+
| user_id | tweet_id | tweet                                                      | tweet_date |
+---------+----------+------------------------------------------------------------+------------+
| 135     | 13       | Enjoying a great start to the day. #HappyDay #MorningVibes | 2024-02-01 |
| 136     | 14       | Another #HappyDay with good vibes! #FeelGood               | 2024-02-03 |
| 137     | 15       | Productivity peaks! #WorkLife #ProductiveDay               | 2024-02-04 |
| 138     | 16       | Exploring new tech frontiers. #TechLife #Innovation        | 2024-02-04 |
| 139     | 17       | Gratitude for today's moments. #HappyDay #Thankful         | 2024-02-05 |
| 140     | 18       | Innovation drives us. #TechLife #FutureTech                | 2024-02-07 |
| 141     | 19       | Connecting with nature's serenity. #Nature #Peaceful       | 2024-02-09 |
+---------+----------+------------------------------------------------------------+------------+
```
**Output:**
``` sql
+-----------+-------+
| hashtag   | count |
+-----------+-------+
| #HappyDay | 3     |
| #TechLife | 2     |
| #WorkLife | 1     |
+-----------+-------+
```
**Explanation:**

#HappyDay: Appeared in tweet IDs 13, 14, and 17, with a total count of 3 mentions.

#TechLife: Appeared in tweet IDs 16 and 18, with a total count of 2 mentions.

#WorkLife: Appeared in tweet ID 15, with a total count of 1 mention.

Note: Output table is sorted in descending order by count and hashtag respectively.

**Solution-1:**
``` sql
with recursive cte as(
SELECT REGEXP_SUBSTR(tweet, '#[a-zA-Z0-9_]+') AS hashtag,
REGEXP_REPLACE(tweet, '#[a-zA-Z]+','',1,1) as tweet, tweet_date
FROM Tweets
UNION ALL
SELECT REGEXP_SUBSTR(tweet, '#[a-zA-Z0-9_]+') AS hashtag,
REGEXP_REPLACE(tweet, '#[a-zA-Z]+','',1,1) as tweet, tweet_date
FROM cte
WHERE position('#' IN tweet) > 0
)

SELECT hashtag, COUNT(*) AS count
FROM CTE
WHERE YEAR(tweet_date) = 2024 AND MONTH(tweet_date) = 2
GROUP BY hashtag
ORDER BY count DESC, hashtag DESC
LIMIT 3



**Solution-2:**
``` sql
WITH RECURSIVE  A AS(
    SELECT SUBSTRING_INDEX(SUBSTRING_INDEX(tweet, "#", -1), " ", 1) AS tag,
    SUBSTRING(tweet, 1, LENGTH(tweet) - LOCATE('#', REVERSE(tweet))) AS remain
    FROM Tweets
    UNION ALL
    SELECT SUBSTRING_INDEX(SUBSTRING_INDEX(remain, "#", -1), " ", 1) AS tag,
    SUBSTRING(remain, 1, LENGTH(remain) - LOCATE('#', REVERSE(remain))) AS remain
    FROM A WHERE LOCATE('#', remain) > 0
)
SELECT CONCAT("#", tag) AS hashtag , COUNT(*) AS count 
FROM A GROUP BY 1
ORDER BY 2 DESC, 1 DESC LIMIT 3